In [1]:
# Install pyrevealed from PyPI (as John would do)
!pip install pyrevealed -q


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


# Tutorial 2: PreferenceEncoder

The `PreferenceEncoder` follows the scikit-learn pattern (fit/transform) to extract latent preference values from user behavior.

**What you'll learn:**
- Feature extraction for ML models (user embeddings)
- Counterfactual predictions (what would user buy at different prices?)
- Personalization and recommendation systems

In [2]:
import numpy as np
from pyrevealed import BehaviorLog, PreferenceEncoder

## Example 1: Basic Fit and Extract Latent Values

User behavior over 5 observations. This data represents a rational consumer responding to price changes.

In [3]:
log = BehaviorLog(
    cost_vectors=np.array([
        [1.0, 3.0],   # Good A cheap, B expensive
        [3.0, 1.0],   # Good A expensive, B cheap
        [2.0, 2.0],   # Equal prices
        [1.5, 2.5],   # A slightly cheaper
        [2.5, 1.5],   # B slightly cheaper
    ]),
    action_vectors=np.array([
        [6.0, 1.0],   # Bought more A (it was cheap)
        [1.0, 6.0],   # Bought more B (it was cheap)
        [3.0, 3.0],   # Equal amounts at equal prices
        [5.0, 2.0],   # More A when A cheaper
        [2.0, 5.0],   # More B when B cheaper
    ]),
    user_id="user_001"
)

# Fit encoder to behavior
encoder = PreferenceEncoder()
encoder.fit(log)

print(f"Encoder fitted: {encoder.is_fitted}")
print(f"Solver status: {encoder.solver_status}")

# Extract latent values (one per observation)
latent_values = encoder.extract_latent_values()
print(f"\nLatent values per observation:")
for i, val in enumerate(latent_values):
    print(f"  Observation {i}: {val:.4f}")

# Extract marginal weights (sensitivity to cost changes)
marginal_weights = encoder.extract_marginal_weights()
print(f"\nMarginal weights (price sensitivity):")
for i, w in enumerate(marginal_weights):
    print(f"  Observation {i}: {w:.4f}")

print(f"\nMean marginal weight: {encoder.mean_marginal_weight:.4f}")

Encoder fitted: True
Solver status: Optimization terminated successfully. (HiGHS Status 7: Optimal)

Latent values per observation:
  Observation 0: 0.0000
  Observation 1: 0.0000
  Observation 2: 0.0000
  Observation 3: 0.0000
  Observation 4: 0.0000

Marginal weights (price sensitivity):
  Observation 0: 0.0000
  Observation 1: 0.0000
  Observation 2: 0.0000
  Observation 3: 0.0000
  Observation 4: 0.0000

Mean marginal weight: 0.0000


/private/tmp/john_test/lib/python3.11/site-packages/pyrevealed/algorithms/utility.py:106: OptimizeWarning: Unrecognized options detected: {'tol': 1e-08}. These will be passed to HiGHS verbatim.
  result = linprog(
/private/tmp/john_test/lib/python3.11/site-packages/scipy/optimize/_linprog_highs.py:355: OptimizeWarning: Unrecognized options detected: {'tol': 1e-08}
  res = _highs_wrapper(c, A.indptr, A.indices, A.data, lhs, rhs,


## Example 2: Value Function for Counterfactuals

Get a callable function that estimates the value of any action bundle.

In [4]:
# Get a callable value function
value_fn = encoder.get_value_function()

# Estimate value of hypothetical bundles
test_bundles = [
    [3.0, 3.0],   # Equal amounts
    [10.0, 0.0],  # Only good A
    [0.0, 10.0],  # Only good B
    [5.0, 5.0],   # More of both
]

print("Estimated values for hypothetical bundles:")
for bundle in test_bundles:
    value = value_fn(np.array(bundle))
    print(f"  Bundle {bundle}: value = {value:.4f}")

Estimated values for hypothetical bundles:
  Bundle [3.0, 3.0]: value = 0.0000
  Bundle [10.0, 0.0]: value = 0.0000
  Bundle [0.0, 10.0]: value = 0.0000
  Bundle [5.0, 5.0]: value = 0.0000


## Example 3: Predict Choice Under New Conditions

What would the user buy if prices changed?

In [5]:
new_prices = np.array([0.5, 2.0])  # Good A becomes very cheap
budget = 10.0

predicted = encoder.predict_choice(new_prices, budget)
if predicted is not None:
    print(f"New prices: {new_prices}")
    print(f"Budget: ${budget}")
    print(f"Predicted purchase: {predicted}")
    print(f"Predicted spend: ${np.dot(new_prices, predicted):.2f}")
else:
    print("Prediction failed (behavior may be too inconsistent)")

# Another scenario
print("\nScenario 2: Good B becomes cheap")
new_prices_2 = np.array([3.0, 0.5])
predicted_2 = encoder.predict_choice(new_prices_2, budget)
if predicted_2 is not None:
    print(f"New prices: {new_prices_2}")
    print(f"Budget: ${budget}")
    print(f"Predicted purchase: {predicted_2}")

New prices: [0.5 2. ]
Budget: $10.0
Predicted purchase: [20.  0.]
Predicted spend: $10.00

Scenario 2: Good B becomes cheap
New prices: [3.  0.5]
Budget: $10.0
Predicted purchase: [ 0. 20.]


## Example 4: ML Feature Extraction

Extract features from multiple users for machine learning models.

In [6]:
# Simulate multiple users
np.random.seed(42)
users = []
features_list = []

for user_idx in range(5):
    # Generate synthetic user data
    T = np.random.randint(5, 10)
    prices = np.abs(np.random.randn(T, 3)) + 0.5
    quantities = np.abs(np.random.randn(T, 3))

    user_log = BehaviorLog(
        cost_vectors=prices,
        action_vectors=quantities,
        user_id=f"user_{user_idx:03d}"
    )

    enc = PreferenceEncoder()
    enc.fit(user_log)

    if enc.is_fitted:
        # Use mean latent value as user-level feature
        mean_latent = np.mean(enc.extract_latent_values())
        mean_marginal = enc.mean_marginal_weight
        users.append(user_log.user_id)
        features_list.append([mean_latent, mean_marginal])

print("Extracted features per user (for ML models):")
print(f"{'User':<12} {'Mean Latent':<14} {'Mean Marginal':<14}")
print("-" * 40)
for user, feats in zip(users, features_list):
    print(f"{user:<12} {feats[0]:<14.4f} {feats[1]:<14.4f}")

Extracted features per user (for ML models):
User         Mean Latent    Mean Marginal 
----------------------------------------
user_004     0.0000         0.0000        


These features can be used for:
- User clustering (segment users by preference patterns)
- Churn prediction (users with unstable preferences)
- Personalization (similar preference = similar recommendations)

## Example 5: Diagnostic Details

Get detailed diagnostics about the fitting process.

In [7]:
details = encoder.get_fit_details()
print(f"LP solver success: {details.success}")
print(f"Solver status: {details.lp_status}")
print(f"Computation time: {details.computation_time_ms:.2f} ms")
print(f"Mean marginal utility: {details.mean_marginal_utility:.4f}")

if details.residuals is not None:
    print(f"Max constraint residual: {np.max(np.abs(details.residuals)):.6f}")

LP solver success: True
Solver status: Optimization terminated successfully. (HiGHS Status 7: Optimal)
Computation time: 14.88 ms
Mean marginal utility: 0.0000
Max constraint residual: 0.000010
